# ConvLSTM_GPT4_001をもとにコードを書く
- パラメーターの調整
- バッチ間の処理を行いたい

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import random
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.modules.loss")

class CFG:
    make_folder = True
    note_num = "StackingOpt/ConvLSTM003"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


class CustomDataset(Dataset):
    def __init__(self, dates, input_dir, transform=None):
        self.input_dir = input_dir
        self.transform = transform

        self.date_list = dates

    def __len__(self):
        return len(self.date_list) 

    def __getitem__(self, idx):
        target_date = self.date_list[idx]
        input_date = (datetime.strptime(self.date_list[idx], "%Y%m%d%H%M") - timedelta(minutes=30)).strftime("%Y%m%d%H%M")

        input_csv = os.path.join(self.input_dir, f"interpolated_mesh_data_{input_date}.csv")
        target_csv = os.path.join(self.input_dir, f"interpolated_mesh_data_{target_date}.csv")

        input_data = pd.read_csv(input_csv,index_col=0).values.reshape(1, 16, 12)
        target_data = pd.read_csv(target_csv,index_col=0).values.reshape(1, 16, 12)

        input_tensor = torch.tensor(input_data, dtype=torch.float32)
        target_tensor = torch.tensor(target_data, dtype=torch.float32)

        if self.transform:
            input_tensor = self.transform(input_tensor)
            target_tensor = self.transform(target_tensor)

        return input_tensor, target_tensor


# 2次元ConvLSTMモデルは先程定義したものを使用
class ConvLSTM2D(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvLSTM2D, self).__init__()

        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2

        self.conv_lstm_cell = nn.Conv2d(input_channels + hidden_channels, 4 * hidden_channels, kernel_size, padding=self.padding)
        
    def forward(self, input_tensor, hidden_state):
        h_cur, c_cur = hidden_state

        combined = torch.cat((input_tensor, h_cur), 1)
        gates = self.conv_lstm_cell(combined)

        input_gate, forget_gate, cell_gate, output_gate = gates.chunk(4, 1)

        input_gate = torch.sigmoid(input_gate)
        forget_gate = torch.sigmoid(forget_gate)
        cell_gate = torch.tanh(cell_gate)
        output_gate = torch.sigmoid(output_gate)

        c_next = forget_gate * c_cur + input_gate * cell_gate
        h_next = output_gate * torch.tanh(c_next)

        return h_next, c_next

def create_data_loaders(input_dir, batch_size, train_dates,valid_dates):
    train_dataset = CustomDataset(train_dates, input_dir)
    valid_dataset = CustomDataset(valid_dates, input_dir)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, valid_loader

def time_series_split(start_date, end_date, input_dir, n_splits):
    start_date = datetime.strptime(start_date, "%Y%m%d%H%M")
    end_date = datetime.strptime(end_date, "%Y%m%d%H%M")
    delta = (end_date - start_date) // n_splits

    splits = []
    for i in range(n_splits):
        train_start = start_date + delta * i
        train_end = train_start + delta
        valid_start = train_end
        valid_end = valid_start + delta

        splits.append(((train_start.strftime("%Y%m%d%H%M"), train_end.strftime("%Y%m%d%H%M")),
                       (valid_start.strftime("%Y%m%d%H%M"), valid_end.strftime("%Y%m%d%H%M"))))

    return splits

def train(model, criterion, optimizer, train_loader, valid_loader, device, epochs, batch_size, hidden_channels, height, width, patience,use_meshnums):
    early_stop_counter = 0
    best_valid_loss = float("inf")

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for input_batch, target_batch in train_loader:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            current_batch_size = input_batch.size(0)
            h, c = torch.zeros(current_batch_size, hidden_channels, height, width).to(device), torch.zeros(current_batch_size, hidden_channels, height, width).to(device)

            optimizer.zero_grad()

            h, c = model(input_batch, (h, c))

            loss = criterion(h.reshape(current_batch_size,1,-1)[:,0,use_meshnums], target_batch.reshape(current_batch_size,1,-1)[:,0,use_meshnums])
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for input_batch, target_batch in valid_loader:
                input_batch, target_batch = input_batch.to(device), target_batch.to(device)

                current_batch_size = input_batch.size(0)
                h, c = torch.zeros(current_batch_size, hidden_channels, height, width).to(device), torch.zeros(current_batch_size, hidden_channels, height, width).to(device)

                h, c = model(input_batch, (h, c))

                loss = criterion(h.reshape(current_batch_size,1,-1)[:,0,use_meshnums], target_batch.reshape(current_batch_size,1,-1)[:,0,use_meshnums])
                valid_loss += loss.item()

        valid_loss /= len(valid_loader)

        if (epoch + 1) % 20 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}')

        # Check for early stopping
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter > patience:
            print(f"Early stopping after {patience} epochs without improvement.")
            break

def save_model(model, save_path, fold):
    torch.save(model.state_dict(), save_path.format(fold))

def load_model(model, load_path, device):
    model.load_state_dict(torch.load(load_path, map_location=device))
    model.to(device)

def inference(model, test_loader, device, height, width, hidden_channels):
    model.eval()
    test_preds = []

    with torch.no_grad():
        for input_batch, _ in test_loader:
            input_batch = input_batch.to(device)

            current_batch_size = input_batch.size(0)
            h, c = torch.zeros(current_batch_size, hidden_channels, height, width).to(device), torch.zeros(current_batch_size, hidden_channels, height, width).to(device)

            h, c = model(input_batch, (h, c))

            test_preds.append(h.cpu().numpy())

    test_preds = np.concatenate(test_preds, axis=0)

    return test_preds

def compute_mse(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    return mse

def prepare_data_loaders(input_dir, batch_size, train_dates, valid_dates, test_dates):
    train_loader, valid_loader = create_data_loaders(input_dir, batch_size, train_dates, valid_dates)
    _, test_loader = create_data_loaders(input_dir, batch_size, test_dates, test_dates)
    return train_loader, valid_loader, test_loader


def train_and_evaluate_fold(model, device, num_epochs, batch_size, hidden_channels, height, width, patience,
                            input_dir, train_dates, valid_dates, test_dates, fold, criterion, optimizer,save_path,use_meshnums):
    train_loader, valid_loader, test_loader = prepare_data_loaders(input_dir, batch_size, train_dates, valid_dates, test_dates)

    train(model, criterion, optimizer, train_loader, valid_loader, device, num_epochs, batch_size, hidden_channels, height, width, patience,use_meshnums)

    save_model(model, save_path, fold)
    
    valid_true = np.concatenate([target_batch.numpy() for _, target_batch in valid_loader], axis=0)
    valid_preds = inference(model, valid_loader, device, height, width, hidden_channels)
    #print(valid_preds.shape)

    valid_mse = compute_mse(valid_true.reshape(valid_true.shape[0],1,-1)[:,0,use_meshnums], valid_preds.reshape(valid_preds.shape[0],1,-1)[:,0,use_meshnums])
    print(f"  Valid MSE for fold {fold}: {valid_mse:.4f}")

    test_preds = inference(model, test_loader, device, height, width, hidden_channels)
    test_true = np.concatenate([target_batch.numpy() for _, target_batch in test_loader], axis=0)

    return valid_true, valid_preds, test_true, test_preds

def create_time_series_data(start_date, end_date):

    start_date = datetime.strptime(start_date, "%Y%m%d%H%M")
    end_date = datetime.strptime(end_date, "%Y%m%d%H%M")
    delta = timedelta(minutes=30)

    data_list = []
    date = start_date
    while date <= end_date:
        if date.hour > 6 and date.hour < 18:
            data_list.append(date.strftime("%Y%m%d%H%M"))
        elif date.hour == 18 and date.minute == 0:
            data_list.append(date.strftime("%Y%m%d%H%M"))
        date += delta
    
    return data_list

def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ハイパーパラメータの設定
    batch_size = 64
    input_channels = 1
    hidden_channels = 1
    kernel_size = (3, 3)
    height, width = 16, 12
    num_epochs = 600
    patience = 20
    learning_rate = 1e-3
    weight_decay = 1e-5

    # 入力ディレクトリと日付の設定
    input_dir = "H:\study\output\StackingOpt\EDA005"
    train_start_date = "201406010000"
    train_end_date = "201407010000"
    test_start_date = "201407010000"
    test_end_date = "201407310000"

    # 時系列の分割設定
    n_splits = 3
    train_data_list = create_time_series_data(train_start_date,train_end_date)
    train_date_list_split = np.array_split(train_data_list, n_splits)

    test_dates = create_time_series_data(test_start_date,test_end_date)


    # 保存先ディレクトリの設定
    save_path = OUTPUT_DIR + "/saved_model_fold_{}.pth"


    # 変数の初期化
    oof_preds = None
    oof_true = None
    test_preds_ensemble = None

    #誤差計算に使うメッシュデータの番号
    use_meshnums = np.load(input_dir.replace("\\","/")+"/use_meshnums.npy").tolist()

    set_seeds()
    for fold in range(len(train_date_list_split)):
        print(f"\nFold {fold + 1}")
        train_dates = np.concatenate(train_date_list_split[:fold] + train_date_list_split[fold+1:])
        valid_dates = train_date_list_split[fold] 


        model = ConvLSTM2D(input_channels, hidden_channels, kernel_size).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        valid_true, valid_preds, test_true, test_preds = train_and_evaluate_fold(
            model, device, num_epochs, batch_size, hidden_channels, height, width, patience,
            input_dir, train_dates, valid_dates, test_dates, fold, criterion, optimizer,save_path.format(fold + 1),use_meshnums
        )

        if oof_preds is None:
            oof_preds = valid_preds
            oof_true = valid_true
            oof_dates = valid_dates
            test_preds_ensemble = test_preds
        else:
            oof_preds = np.vstack([oof_preds, valid_preds])
            oof_true = np.vstack([oof_true, valid_true])
            oof_dates = np.hstack([oof_dates, valid_dates])
            test_preds_ensemble += test_preds


    test_preds_ensemble /= n_splits
    oof_mse = compute_mse(oof_true.reshape(oof_true.shape[0],1,-1)[:,0,use_meshnums], oof_preds.reshape(oof_preds.shape[0],1,-1)[:,0,use_meshnums])
    test_mse = compute_mse(test_true.reshape(test_true.shape[0],1,-1)[:,0,use_meshnums], test_preds_ensemble.reshape(test_preds.shape[0],1,-1)[:,0,use_meshnums])
    print(f"\nOverall Out-of-Fold MSE: {oof_mse:.4f}")
    print(f"\nOverall Test MSE: {test_mse:.4f}")

    np.save(f"{OUTPUT_DIR}/oof_preds.npy", oof_preds)
    np.save(f"{OUTPUT_DIR}/oof_true.npy", oof_true)
    np.save(f"{OUTPUT_DIR}/test_preds_ensemble.npy", test_preds_ensemble)
    np.save(f"{OUTPUT_DIR}/test_true.npy", test_true)

    np.save(f"{OUTPUT_DIR}/oof_dates.npy", oof_dates)
    np.save(f"{OUTPUT_DIR}/test_dates_ensemble", test_dates)

if __name__ == "__main__":
    main()



Fold 1
Epoch [20/600], Train Loss: 0.1039, Valid Loss: 0.0854
Epoch [40/600], Train Loss: 0.0542, Valid Loss: 0.0437
Epoch [60/600], Train Loss: 0.0424, Valid Loss: 0.0336
Epoch [80/600], Train Loss: 0.0381, Valid Loss: 0.0304
Epoch [100/600], Train Loss: 0.0371, Valid Loss: 0.0293
Epoch [120/600], Train Loss: 0.0369, Valid Loss: 0.0288
Epoch [140/600], Train Loss: 0.0376, Valid Loss: 0.0286
Epoch [160/600], Train Loss: 0.0369, Valid Loss: 0.0284
Epoch [180/600], Train Loss: 0.0366, Valid Loss: 0.0283
Epoch [200/600], Train Loss: 0.0361, Valid Loss: 0.0281
Epoch [220/600], Train Loss: 0.0369, Valid Loss: 0.0280
Epoch [240/600], Train Loss: 0.0360, Valid Loss: 0.0278
Epoch [260/600], Train Loss: 0.0361, Valid Loss: 0.0277
Epoch [280/600], Train Loss: 0.0360, Valid Loss: 0.0276
Epoch [300/600], Train Loss: 0.0367, Valid Loss: 0.0275
Epoch [320/600], Train Loss: 0.0354, Valid Loss: 0.0274
Epoch [340/600], Train Loss: 0.0368, Valid Loss: 0.0273
Epoch [360/600], Train Loss: 0.0357, Valid L

# 出力確認用

In [3]:
oof_dates = np.load(f"{OUTPUT_DIR}/oof_dates.npy")
oof_true = np.load(f"{OUTPUT_DIR}/oof_true.npy")
oof_preds = np.load(f"{OUTPUT_DIR}/oof_preds.npy")

In [5]:
scores = []
for i in range(len(oof_dates)):
    mesh = pd.read_csv(f"H:\study\output\StackingOpt\EDA005\interpolated_mesh_data_{oof_dates[i]}.csv",index_col=0)
    t = pd.DataFrame(oof_true[i][0]).to_numpy().reshape(-1)-mesh.to_numpy().reshape(-1)
    scores.append(t.sum())
    

In [6]:
np.min(scores),np.max(scores),np.sum(scores)

(-5.424880985538394e-07, 5.17612458528216e-06, 0.00010159391689251902)

In [7]:
oof_mse = compute_mse(oof_true, oof_preds)
oof_mse

0.028993767

In [8]:
test_preds_ensemble = np.load(f"{OUTPUT_DIR}/test_preds_ensemble.npy")
test_dates = np.load(f"{OUTPUT_DIR}/test_dates_ensemble.npy")

test_true = np.load(f"{OUTPUT_DIR}/test_true.npy")

In [11]:
score = []
mesh_list = []
for i in range(len(test_dates)):
    mesh = pd.read_csv(f"H:\study\output\StackingOpt\EDA005\interpolated_mesh_data_{test_dates[i]}.csv",index_col=0)
    mesh = mesh.to_numpy().reshape(1, 16, 12)
    test = test_true[i].reshape(1, 16, 12)
    score.append(np.sum(mesh-test))
    mesh_list.append(mesh)

test_true_mesh = np.concatenate(mesh_list, axis=0)    

print(np.min(scores),np.max(scores),np.sum(scores))

test_mse = compute_mse(test_true, test_preds_ensemble)
test_mse_mesh = compute_mse(test_true_mesh, test_preds_ensemble)
test_mse-test_mse_mesh 

-5.424880985538394e-07 5.17612458528216e-06 0.00010159391689251902


-0.09397567092087489

In [12]:
test_preds_ensemble = np.load(f"{OUTPUT_DIR}/test_preds_ensemble.npy")

In [13]:
test_preds_ensemble[0].shape

(1, 16, 12)

# テストデータの整理

In [14]:
test_preds_ensemble = np.load(f"{OUTPUT_DIR}/test_preds_ensemble.npy")
test_dates = np.load(f"{OUTPUT_DIR}/test_dates_ensemble.npy")

test_true = np.load(f"{OUTPUT_DIR}/test_true.npy")

In [15]:
lat = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\lati_zenkoku.csv", header=None, index_col=None)
lon = pd.read_csv(r"H:\study\preprocessing_data\3_mesh_place\long_zenkoku.csv", header=None, index_col=None)
id_all_data = pd.read_csv(r"H:\study\preprocessing_data\id_all_data.csv", encoding="shift_jis")

unique_id = ['10000095', '10000269', '1020000002', '1110000001', '1110000010', '1110000011', '1110000012', '1110000013', '1110000014', '1110000015', '1160000025', '1160000090', '1160000091', '1160000182', '1160000185', '1160000253', '1160000387', '1160000402', '1160000419', '1160000420', '1160000423', '1270000026', '1280000048', '1550000001', '1650000004', '1680000001', '1680000002', '1680000003', '1680000004', '1680000010', '1680000017', '1680000021', '1680000033', '1680000047', '1680000054', '1680000057', '1680000063', '1680000067', '1680000080', '1680000081', '1680000097', '1680000107', '1680000108', '1680000112', '1680000151', '1680000152', '1680000213', '1680000216', '1680000217', '1680000218', '1680000223', '1680000228', '1680000285', '1680000287', '1680000327', '1680000364', '2220000001', '2220000002', '2220000003', '2730000001', '2910000002', '3000000007', '3000000012', '3000000042', '5000000044', '5000000045', '6000000016', '6000000017', '6060000016', '6060000017', '6060000018', '6170000016', '6170000123', '6170000124', '6170000125', '6620000065', '6620000066', '6620000088', '6620000089', '6620000111', '6620000117', '6620000118', '6620000121', '6620000122', '6620000123', '6620000124', '6620000131', '6620000132', '6910000180', '6910000198', '6910000200', '6910000206', '6910000216', '6910000217', '6910000239', '6910000240', '6910000249', '6910000250', '6910000421', '6910000424', '6910000425', '6910000469', '6910000470']
unique_id = [int(i) for i in unique_id]
to_unique_id = [str(num).zfill(10) for num in unique_id]
id_data = id_all_data[id_all_data.id.isin(unique_id)].reset_index(drop=True)

min_lat,max_lat = id_data.id_lat.min(),id_data.id_lat.max()
min_lng,max_lng = id_data.id_lng.min(),id_data.id_lng.max()
userow = (lon.iloc[:,0]>=min_lng)&(lon.iloc[:,0]<=max_lng)
usecol = (lat.iloc[0]>=min_lat)&(lat.iloc[0]<=max_lat)

lat = lat.loc[userow,usecol]
lon = lon.loc[userow,usecol]

In [16]:
preds = test_preds_ensemble.reshape(-1)
dates_np = np.repeat(test_dates, lat.shape[0]*lat.shape[1])

lat_np = lat.to_numpy().reshape(-1)
lat_np_repeated = np.tile(lat_np, (len(test_dates)))

lon_np = lon.to_numpy().reshape(-1)
lon_np_repeated = np.tile(lon_np, (len(test_dates)))

In [17]:
preds = pd.DataFrame(zip(dates_np,lat_np_repeated,lon_np_repeated,preds),columns=["datetime","id_lat_mesh","id_lng_mesh","pred"])
preds["datetime"] = pd.to_datetime(preds["datetime"], format='%Y%m%d%H%M')

In [18]:
preds

,datetime,id_lat_mesh,id_lng_mesh,pred
0,2014-07-01 07:00:00,35.66,139.90,0.271133
1,2014-07-01 07:00:00,35.68,139.90,0.384261
2,2014-07-01 07:00:00,35.70,139.90,0.387107
3,2014-07-01 07:00:00,35.72,139.90,0.399533
4,2014-07-01 07:00:00,35.74,139.90,0.424877
...,...,...,...,...
124195,2014-07-30 18:00:00,35.80,140.18,0.735436
124196,2014-07-30 18:00:00,35.82,140.18,0.736351
124197,2014-07-30 18:00:00,35.84,140.18,0.736787
124198,2014-07-30 18:00:00,35.86,140.18,0.736174


In [19]:
import utils 

df = utils.get_preprocessing_data(to_unique_id)
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.merge(id_data[["id","id_lat_mesh","id_lng_mesh","pvrate","observed_max"]],on=["id"],how="left")

date_range = pd.to_datetime(test_dates, format='%Y%m%d%H%M')
df = df[df.datetime.isin(date_range)]

In [20]:
df = df.merge(preds,on=["datetime","id_lat_mesh","id_lng_mesh"],how="left")

df["pred*two_weeks_max"] = df["pred"]*df["two_weeks_max"]
df["nv*twoweeks_max"] = df["nv"]*df["two_weeks_max"]

In [21]:
df["APE"] = np.abs(df["pred*two_weeks_max"]-df["nv*twoweeks_max"])/df["observed_max"]*100

In [22]:
df["APE"].mean()

16.106519847307815

In [20]:
y_true = df["pred*two_weeks_max"] 
y_pred = df["nv*twoweeks_max"]
np.sqrt(np.mean((y_true - y_pred)**2))

1223.4059527433953

In [23]:
y_true = df["pred"]
y_pred = df["nv"]
np.mean((y_true - y_pred)**2)

0.045501480973918575